In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.typing import NDArray

In [4]:
v = 1
beta = 4
dy = 0.05
dx = 0.02 # tau
theta = 3/4
#TODO: klopt dit?
n = int(1 / dy - 1) # FIXME: moet error geven voor float
x_lst: NDArray[np.floating] = dx * np.array([5, 10, 15, 20, 25]) #FIXME: moet int arr zijn, hangt af van dx, geef anders error
num_iterations = x_lst[-1] // dx
x = x_lst[0]


def create_matrix_A(*, n: int = n, v: float = v, dy: float = dy, beta: float = beta):
    factor1 = -1 / (v * (dy**2))
    factor2 = -beta**2 / v

    main_diag = 2 * np.ones(n + 1)
    off_diag = -1 * np.ones(n)
    
    matrix1 = np.diag(main_diag) + np.diag(off_diag, k=1) + np.diag(off_diag, k=-1)
    matrix2 = np.eye(n + 1)

    A = factor1 * matrix1 + factor2 * matrix2
    A[0, 1] = -2 # Neumann boundary condition at y=0
    return A

def create_r(*, n: int = n):
    r = np.zeros(n + 1) # r is a constant as it does not depend on x
    #TODO: klopt dit?
    r[-1] = 1 / dy**2 # Dirichlet boundary condition at y=1
    return r

def create_u_0(*, n: int = n):
    u_0 = np.zeros(n + 1) # Dirichlet boundary condition at x=0
    u_0[-1] = 1 # Dirichlet boundary condition at y=1
    # TODO: wel of niet 1 gebruiken?
    return u_0

A = create_matrix_A()
r = create_r()
u_0 = create_u_0()







def theta_method(
    *,
    c_n: NDArray[np.float64],
    A: NDArray[np.float64] = A,
    r: NDArray[np.float64] = r,
    dx: float = dx,
    theta: float = theta
):
    c_next = np.linalg.solve()
    return c_next


for n in n_lst:
    c = np.zeros(n_y) 
    for _ in range(n):
        c = theta_method(c, A, r, tau, theta)
    solutions.append(c.copy())
    alpha = (3*c[-1] - 4*c[-2] + c[-3]) / (2*dy)  # Second-order backward difference
    alphas.append(alpha)
    
# Plotting
plt.figure(figsize=(10, 6))
for i, solution in enumerate(solutions):
    plt.plot(y, solution, label=f'x = {n_lst[i] * tau:.2f}')
plt.xlabel('y')
plt.ylabel('c (y)')
plt.title('Numerical Solution of c(y) at different times')
plt.legend()
plt.grid(True)

plt.xlim(0, 2)
plt.ylim(0, max([max(sol) for sol in solutions] + [1])+1)

plt.show()

[[-816.   -2.   -0.]
 [ 400. -816.  400.]
 [  -0.  400. -816.]]


NameError: name 'tau' is not defined